## Requirements

In [325]:
import pandas as pd

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from scipy.spatial import Voronoi, voronoi_plot_2d
import folium # map rendering library

import requests # library to handle requests

#Use geojson file to write out the features
from geojson import FeatureCollection, Feature, Polygon
#Data import
import os

## Load the data
In the case of "data.csv", I formatted the csv an online regex tool.

data.csv :

    Postal Code,Borough,Neighbourhood
    M1A,Not assigned,"Not assigned"
    M2A,Not assigned,"Not assigned"
    M3A,North York,"Parkwoods"
    M4A,North York,"Victoria Village"
    M5A,Downtown Toronto,"Regent Park, Harbourfront"
    ....
    
   

and "data_cords.csv", was provided by coursera.

#### and Merge data
All rows with "not assigned" values are dropped, there are no cases 
where a row has a borough but a Not assigned neighborhood.

All rows without Latitude and Longitude are dropped.

In [364]:
data=pd.read_csv("data.csv")
data_cords=pd.read_csv("Geospatial_Coordinates.csv")
data = data[data["Borough"] != "Not assigned"]

full_data=pd.merge(data,data_cords,on=["Postal Code"])
full_data[:6]

,Postal Code,Borough,Neighborhoods,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242


In [369]:

latitude=43.72
longitude=-79.4

# hiddencell CLIENT_ID 
# @hiddencell
CLIENT_ID = '3AE2MGLFIMETZBRCNW0IDFIUIJEZNPXJDH5YMHOJZFI3UH52'
# hiddencell CLIENT_SECRET 
# @hiddencell
CLIENT_SECRET = 'OPH1TTVI2AMMCGOEEKYYCNVRCHEK3PSXNZDDGXOGKZTZR4OI'
VERSION = '20180605'

fire_station_id='4bf58dd8d48988d12c941735'
police_department_id='4bf58dd8d48988d12c941735'


def foursquare_search(version,client_id=None,client_secret=None,oauth_token=None,ll=None,near=None,radius=250,limit=10,query=None,categoryId=None):
    request="https://api.foursquare.com/v2/venues/search"
    parameters=[]
    if version:
        parameters.append("v={}".format(version))
    if client_id:
        parameters.append("client_id={}".format(client_id))
    if client_secret:
        parameters.append("client_secret={}".format(client_secret))
    if oauth_token:
        parameters.append("oauth_token={}".format(oauth_token))
    if radius:
        parameters.append("radius={}".format(radius))
    if limit:
        parameters.append("limit={}".format(limit))
    if query:
        parameters.append("query={}".format(query))
    if categoryId:
        parameters.append("categoryId={}".format(",".join(categoryId)))
    if ll:
        parameters.append("ll={}".format("{},{}".format(*ll)))
    if near:
        parameters.append("near={}".format(near))
        
    url="?".join((request,'&'.join(param for param in parameters)))
    results = requests.get(url).json().get("response").get("venues")
    
    return pd.DataFrame.from_dict(results)


### Visualize neighborhoods

In [359]:

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(full_data['Latitude'], full_data['Longitude'], full_data['Borough'], full_data['Neighborhoods']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

<a href="https://imgur.com/F4AElq6"><img src="https://i.imgur.com/F4AElq6.png" title="source: imgur.com" /></a>
https://imgur.com/a/8wu14tn

Here to see the render

## Visualize police coverage per police station

Another category can be selected 


for more categories:
https://developer.foursquare.com/docs/build-with-foursquare/categories/



In [360]:

fire_station_id='4bf58dd8d48988d12c941735'
police_department_id='4bf58dd8d48988d12c941735'
CategoryId=[police_department_id]

police_dep = foursquare_search(VERSION,CLIENT_ID,CLIENT_SECRET,radius=40000,limit=100,ll=[latitude,longitude],categoryId=CategoryId)
police_dep_loc_ = police_dep["location"]
police_dep_loc = pd.DataFrame(list(police_dep_loc_))
police_dep_ll = police_dep_loc[["lat","lng"]]


# create map of New York using latitude and longitude values
f_map = folium.Map(location=[latitude, longitude], zoom_start=11)

In [361]:
#Data import
data = os.path.join('data.geojson')
#The output file, to contain the Voronoi diagram we computed:
vorJSON = open('data.geojson', 'w')
point_voronoi_list = []
feature_list = []

#Use scipy to make the voronoi diagram:
vor = Voronoi(police_dep_ll.values, qhull_options='Qbb Qc Qx')
for region in range(len(vor.regions)-1):
    vertex_list = []
    for x in vor.regions[region]:
        if x == -1:
            break;
        else:
            #Get the vertex out of the list, and flip the order for folium:
            vertex = vor.vertices[x]
            vertex = (vertex[1], vertex[0])
        vertex_list.append(vertex)
    #Save the vertex list as a polygon and then add to the feature_list:
    polygon = Polygon([vertex_list])
    feature = Feature(geometry=polygon, properties={})
    feature_list.append(feature)

    #Write the features to the new file:
feature_collection = FeatureCollection(feature_list)
print (feature_collection, file=vorJSON)
vorJSON.close()


#Add Voronoid diagram to map object
folium.GeoJson(data, name='Vor').add_to(f_map)

In [362]:



# add markers to map
for ll in police_dep_ll.values:
    label = '{}, {}'.format(ll[0], ll[1])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [ll[0], ll[1]],
        radius=6,
        popup="Police Department",
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(f_map)  

# Neighbourhoods
for lat, lng, borough, neighborhood in zip(full_data['Latitude'], full_data['Longitude'], full_data['Borough'], full_data['Neighborhoods']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(f_map)  

## Ideal area of police coverage per police station
( If all neighborhood had the same population density )

In [363]:
f_map

<a href="https://imgur.com/peawLUh"><img src="https://i.imgur.com/peawLUh.png" title="source: imgur.com" /></a>
https://imgur.com/a/8wu14tn

Here to see the render